## Import libraries

In [1]:
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
from keras.preprocessing.sequence import pad_sequences
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam, Adadelta
from keras.models import model_from_json
from keras import backend as K
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization

from sklearn.metrics import classification_report, confusion_matrix
from textblob import TextBlob

import numpy as np, string, pickle, warnings, random
import matplotlib.pyplot as plt

from IPython.display import display, HTML

from deepexplain.tensorflow import DeepExplain
import tensorflow as tf
warnings.filterwarnings("ignore")

Using TensorFlow backend.
C:\Users\rxjos\anaconda3\envs\xai\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\rxjos\anaconda3\envs\xai\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\rxjos\anaconda3\envs\xai\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\rxjos\anaconda3\envs\xai\lib\site-packages\t

## Download data

In [3]:
topWords = 50000
maxWords = 200
downloadFlag = 0

df = pd.read_csv('./data/multidogoSentLevel/airline/train.tsv', sep='\t')
x_train = df['utterance']
y_train = df['intent']

df = pd.read_csv('./data/multidogoSentLevel/airline/dev.tsv', sep='\t')
x_val = df['utterance']
y_val = df['intent']

df = pd.read_csv('./data/multidogoSentLevel/airline/test.tsv', sep='\t')
x_test = df['utterance']
y_test = df['intent']

classes_train = set(y_train)
classes_val = set(y_val)
classes_test = set(y_test)
classes = classes_train.union(classes_val, classes_test)
nb_classes = len(classes)
sentimentDict = {}
for i, cl in enumerate(classes):
    sentimentDict[i] = cl
sentimentDict2 = {v: k for k, v in sentimentDict.items()}
print(sentimentDict)
y_train = [sentimentDict2[cl] for cl in list(y_train)]
y_val = [sentimentDict2[cl] for cl in list(y_val)]
y_test = [sentimentDict2[cl] for cl in list(y_test)]

data = x_train.append(x_val)
data = data.append(x_test)
vocab = []
for utter in data:
    for word in str(utter).split(' '):
        vocab.append(word)
vocab = set(vocab)
word2Index = {}
for i, word in enumerate(vocab):
    word2Index[word] = i
index2Word = {v: k for k, v in word2Index.items()}

train = []
for utter in x_train:
    utt = [word2Index[word] for word in str(utter).split(' ')]
    train.append(utt)
x_train = train
val = []
for utter in x_val:
    utt = [word2Index[word] for word in str(utter).split(' ')]
    val.append(utt)
x_val = val
test = []
for utter in x_test:
    utt = [word2Index[word] for word in str(utter).split(' ')]
    test.append(utt)
x_test = test

{0: 'confirmation', 1: 'bookflight', 2: 'rejection', 3: 'outofdomain', 4: 'changeseatassignment', 5: 'getseatinfo<div>changeseatassignment', 6: 'closinggreeting', 7: 'contentonly', 8: 'getboardingpass', 9: 'openinggreeting', 10: 'getboardingpass<div>changeseatassignment', 11: 'thankyou', 12: 'getseatinfo'}


In [4]:
print(data)

0                                         hi good morning
1                       i want to book a ticket in flight
2                                                   viswa
3                                       mumbai to chennai
4                                                   92018
                              ...                        
4062                              i am need boarding pass
4063    my name is pandi booking confirmation number p...
4064                                                sorry
4065                                   pandi 154 gmailcom
4066                                                   ok
Name: utterance, Length: 20472, dtype: object


In [5]:
stopWords = []

def getWordsFromIndexList(indexList):
    wordList = []
    for index in indexList:
        wordList.append(index2Word[index])

    return " ".join(wordList)

def getSentiment(predictArray):
    pred = int(predictArray[0])
    return sentimentDict[pred]

def getIndexFromWordList(wordList):
    indexList = []
    for word in wordList:
        print(word)
        indexList.append(str(word2Index[word]))
        
    return indexList

In [6]:
print (len(word2Index))

5279


## Preprocess data

In [7]:
stopIndexList = []

# for stopWord in stopWords:
#     stopIndexList.append(word2Index[stopWord])

trainData = []

for indexList in x_train:
    processedList = [index for index in indexList]# if index not in stopIndexList]
    trainData.append(processedList)
    
x_train = trainData

## Data Pre-processing

In [8]:
'''
Padding data to keep vectors of same size
If size < 100 then it will be padded, else it will be cropped
'''
trainX = pad_sequences(x_train, maxlen = maxWords, value = 0.)
valX = pad_sequences(x_val, maxlen = maxWords, value = 0.)
testX = pad_sequences(x_test, maxlen = maxWords, value = 0.)

'''
One-hot encoding for the classes
'''
trainY = np_utils.to_categorical(y_train, num_classes = nb_classes)
valY = np_utils.to_categorical(y_val, num_classes = nb_classes)
testY = np_utils.to_categorical(y_test, num_classes = nb_classes)

## Network Parameters

In [9]:
p_W, p_U, weight_decay = 0, 0, 0
regularizer = l2(weight_decay) if weight_decay else None
sgdOptimizer = 'adam'
lossFun='categorical_crossentropy'
batchSize=25
numEpochs = 5
numHiddenNodes = 256

## Network Architecture

In [10]:
model = Sequential()
model.add(Embedding(topWords, numHiddenNodes, name='embedding_layer'))

model.add(Bidirectional(LSTM(numHiddenNodes, return_sequences=True,
                            recurrent_regularizer=regularizer, kernel_regularizer=regularizer,
                            bias_regularizer=regularizer, recurrent_dropout=p_W, dropout=p_U),
                        merge_mode='concat', name='bidi_lstm_layer'))
model.add(LSTM(numHiddenNodes, name = 'lstm_layer'))
model.add(Dropout(0.5, name = 'dropout'))

model.add(Dense(numHiddenNodes, name='dense_1'))
model.add(Dense(nb_classes, name='dense_2'))
model.add(Activation("softmax"))
# adam = Adam(lr=0.0001)
#adadelta = Adadelta(lr=0.01, rho=0.95, epsilon=1e-08)
model.compile(loss=lossFun, optimizer=sgdOptimizer, metrics=["accuracy"])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, None, 256)         12800000  
_________________________________________________________________
bidi_lstm_layer (Bidirection (None, None, 512)         1050624   
_________________________________________________________________
lstm_layer (LSTM)            (None, 256)               787456    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 13)                3341      
_________________________________________________________________
activation_1 (Activation)    (None, 13)                0         
Total para

## Training the model

In [11]:
model.fit(trainX, trainY, batch_size=batchSize, epochs=numEpochs, verbose=1, validation_data=(valX, valY))

Train on 14354 samples, validate on 2051 samples
Epoch 1/5
14354/14354 [==============================] - 1001s - loss: 0.6730 - acc: 0.8056 - val_loss: 0.4174 - val_acc: 0.8854
Epoch 2/5
14354/14354 [==============================] - 1049s - loss: 0.3439 - acc: 0.9053 - val_loss: 0.3714 - val_acc: 0.8971
Epoch 3/5
14354/14354 [==============================] - 1106s - loss: 0.2891 - acc: 0.9157 - val_loss: 0.3907 - val_acc: 0.9025
Epoch 4/5
14354/14354 [==============================] - 1211s - loss: 0.2703 - acc: 0.9209 - val_loss: 0.3889 - val_acc: 0.8986
Epoch 5/5
14354/14354 [==============================] - 1273s - loss: 0.2587 - acc: 0.9249 - val_loss: 0.3712 - val_acc: 0.8996


# Model accuracy

In [12]:
score = model.evaluate(testX, testY, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

acc: 90.66%


In [13]:
predY = model.predict_classes(testX)
yPred = np_utils.to_categorical(predY, num_classes = nb_classes)
print("Classification Report:\n")
print(classification_report(testY, yPred))

4067/4067 [==============================] - 53s    
Classification Report:

              precision    recall  f1-score   support

           0       0.84      0.92      0.88       640
           1       0.90      0.67      0.76       117
           2       0.88      0.95      0.92       332
           3       0.71      0.55      0.62       133
           4       0.87      0.85      0.86       349
           5       0.00      0.00      0.00         1
           6       0.94      0.89      0.92       110
           7       0.96      0.95      0.95      1370
           8       0.96      0.96      0.96       185
           9       0.97      0.99      0.98       473
          10       0.00      0.00      0.00         0
          11       0.88      0.88      0.88       311
          12       0.40      0.39      0.40        46

   micro avg       0.91      0.91      0.91      4067
   macro avg       0.72      0.69      0.70      4067
weighted avg       0.91      0.91      0.90      4067
 sa

## Save the Tensorflow Model

In [16]:
'''
Serialize model to JSON
'''
model_json = model.to_json()
with open("models/imdb_bi_lstm_big_tensorflow_model_ALL.json", "w") as json_file:
    json_file.write(model_json)

'''
Serialize weights to HDF5
'''
model.save_weights("models/imdb_bi_lstm_big_tensorflow_model_ALL.h5", overwrite=True)
print("Saved model to disk...")

FileNotFoundError: [Errno 2] No such file or directory: 'models/imdb_bi_lstm_big_tensorflow_model_ALL.json'

## Load the Tensorflow Model

In [ ]:
'''
Load json and create model
'''
modelNum=1
if modelNum is 1:
    json_file = open('models/imdb_bi_lstm_big_tensorflow_model.json', 'r')
elif modelNum is 2:
    json_file = open('models/imdb_bi_lstm_big_tensorflow_model_NOT.json', 'r')
elif modelNum is 3:
    json_file = open('models/imdb_bi_lstm_big_tensorflow_model_ALL.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

'''
Load weights into new model
'''
if modelNum is 1:
    loaded_model.load_weights("models/imdb_bi_lstm_big_tensorflow_model.h5")
elif modelNum is 2:
    loaded_model.load_weights("models/imdb_bi_lstm_big_tensorflow_model_NOT.h5")
elif modelNum is 3:
    loaded_model.load_weights("models/imdb_bi_lstm_big_tensorflow_model_ALL.h5")

print ("Loading model from disk...")
model = loaded_model
print(model.summary())

## Testing the model

In [14]:
num = 120
num_next = num + 1
print("Testing for test case..." + str(num))
groundTruth = testY[num]
print(groundTruth)
print(len(testY))
print

sampleX = testX[num:num_next]
predictionClass = model.predict_classes(sampleX, verbose=0)
prediction = np_utils.to_categorical(predictionClass, num_classes = nb_classes)[0]

print("Text: " + str(getWordsFromIndexList(x_test[num-1])))
print("\nPrediction: " + str(getSentiment(predictionClass)))
if np.array_equal(groundTruth,prediction):
    print("\nPrediction is Correct")
else:
    print("\nPrediction is Incorrect")

Testing for test case...120
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
4067
Text: hai

Prediction: contentonly

Prediction is Correct


# Explainability

## Get Relevance for each word

In [15]:
def getWordsAndRelevances(sampleX, relevances, prediction):

    unknownIndex = 0 # Index of padding
    indexList = np.ndarray.tolist(sampleX)[0]
    wordList = []
    wordRelevanceList = []
    polarityHighThreshold = 0.4
    polarityLowThreshold = 0.0
    
    # Find word-wize relevance and normalize
    wordRelevances = np.sum(relevances, -1)
    wordRelList = np.ndarray.tolist(wordRelevances)[0]
    
    for i in range(maxWords):
        index = indexList[i]
        relevance = wordRelList[i]

        if index is not unknownIndex:
            word = index2Word[index]
            blobText = TextBlob(word)
            polarity = blobText.sentiment.polarity
            prediction = int(prediction)
#             if prediction is 0 and polarity < -0.4 or prediction is 1 and polarity > 0.4:
            if abs(polarity) > polarityHighThreshold:
                if prediction is 0 and polarity < 0 or prediction is 1 and polarity > 0:
                    relevance = abs(polarity)
                elif prediction is 0 and polarity > 0 or prediction is 1 and polarity < 0:
                    relevance = -1 * abs(polarity)
                    
                print(word, polarity)
                print(word, relevance)
            elif abs(polarity) > polarityLowThreshold:
                if relevance < 0 and polarity > 0 or relevance > 0 and polarity < 0:
                    relevance = polarity
#                     print("Here...")
#                     print(word, polarity)
#                     print(word, relevance)

            wordList.append(index2Word[index])
            wordRelevanceList.append(relevance)

    
    normalizedRelevanceList = [float(rel)/max(map(lambda x: abs(x), wordRelevanceList)) for rel in wordRelevanceList]
    
    return wordList, wordRelevanceList, normalizedRelevanceList

def showWordRelevances(wordList, wordRelevanceList, normalizedRelevanceList):

    print("\nWord Relevances:\n")
    print("\nOriginal Relevance:\n")
    for i in range(len(wordList)):
        word = str(wordList[i])
        # originalRelevance = "{:8.2f}".format(wordRelevanceList[i])
        originalRelevance = wordRelevanceList[i]
        print ("\t\t\t" + str(originalRelevance) + "\t" + word)

    print("\nNormalized Relevance:\n")
    for i in range(len(wordList)):
        word = str(wordList[i])
        normalizedRelevance = "{:8.2f}".format(normalizedRelevanceList[i])
        print ("\t\t\t" + str(normalizedRelevance) + "\t" + word)
        

## Pre-process and get prediction

In [16]:
def removePunct(inputStr):
    
    # Remove punctuations.
    # punctuations = string.punctuation
    # inputStr = ''.join(ch for ch in inputStr if ch not in punctuations)
    inputStr = inputStr.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    inputList = inputStr.lower().split()
    return inputList

def removeStopWords(inputList):
    processedList = [word for word in inputList if word not in stopWords]
    return processedList

def preProcessQuery(inputStr):
    return removeStopWords(removePunct(inputStr))
#     return removePunct(inputStr)

def getIndexArray(inputStr):
    
    words = preProcessQuery(inputStr)
    # print(words)
    wordIndexList = np.array([word2Index[word] if word in word2Index else 0 for word in words])
    wordIndexArray = np.array([wordIndexList], np.int32)
    wordIndexArray = pad_sequences(wordIndexArray, maxlen = maxWords, value = 0.)
    return wordIndexArray

def getPrediction(inputStr):
    
    wordIndexArray = getIndexArray(inputStr)
    predictionScore = model.predict(wordIndexArray[0:1], verbose=0)
    # print (predictionScore)
    prediction = model.predict_classes(wordIndexArray[0:1], verbose=0)
    return prediction, predictionScore

## Get Polarity from input text

In [17]:
inputStr = input("Enter a sentence to test polarity: ")
prediction, predictionScore = getPrediction(inputStr)
print("\nPrediction: " + str(getSentiment(prediction)))
print(predictionScore)

Enter a sentence to test polarity: hai

Prediction: openinggreeting
[[9.9494691e-06 5.1585845e-05 9.0007943e-06 1.3750526e-05 7.5779039e-06
  5.1860445e-07 6.3664908e-07 5.5288168e-04 5.1804068e-06 9.9933428e-01
  3.2776859e-07 2.1712301e-06 1.2113683e-05]]


# Relevance and HeatMap

## Create DeepExplain Session to get word relevances

In [18]:
with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context
    
    '''
    Need to reconstruct the graph in DeepExplain context, using the same weights.
    1. Get the input tensor
    2. Get embedding tensor
    3. Target the output of the last dense layer (pre-softmax)
    '''
    
    inputTensor = model.layers[0].input
    embeddingTensor = model.layers[0].output
    preSoftmax = model.layers[-2].output
    
    # Sample Data for attribution
    wordIndexArray = getIndexArray(inputStr)
    sampleX = pad_sequences(wordIndexArray, maxlen = maxWords, value = 0.)
    
    # Perform Embedding Lookup
    getEmbeddingOutput = K.function([inputTensor],[embeddingTensor])
    embeddingOutput = getEmbeddingOutput([sampleX])[0]
    
    # Get Prediction for attribution
    prediction, predictionScore = getPrediction(inputStr)
    ys = np_utils.to_categorical(prediction, num_classes = nb_classes)
    
    #relevances = de.explain('grad*input', targetTensor * ys, inputTensor, sampleX)
    #relevances = de.explain('saliency', targetTensor * ys, inputTensor, sampleX)
    #relevances = de.explain('intgrad', targetTensor * ys, inputTensor, sampleX)
    #relevances = de.explain('deeplift', targetTensor * ys, inputTensor, sampleX)
    relevances = de.explain('elrp', preSoftmax * ys, embeddingTensor, embeddingOutput)
    #relevances = de.explain('occlusion', targetTensor * ys, inputTensor, sampleX)
    print ("Feature Relevances Generated...")

Feature Relevances Generated...


## Show Relevance

In [19]:
wordList, wordRelevanceList, normalizedRelevanceList = getWordsAndRelevances(sampleX, relevances, prediction[0])
showWordRelevances(wordList, wordRelevanceList, normalizedRelevanceList)


Word Relevances:


Original Relevance:

			3.292778491973877	hai

Normalized Relevance:

			    1.00	hai


## Generate Heatmap

In [20]:
def rescale_score_by_abs (score, max_score, min_score):
    """
    rescale positive score to the range [0.5, 1.0], negative score to the range [0.0, 0.5],
    using the extremal scores max_score and min_score for normalization
    """
    
    # CASE 1: positive AND negative scores occur --------------------
    if max_score>0 and min_score<0:
    
        if max_score >= abs(min_score):   # deepest color is positive
            if score>=0:
                return 0.5 + 0.5*(score/max_score)
            else:
                return 0.5 - 0.5*(abs(score)/max_score)

        else:                             # deepest color is negative
            if score>=0:
                return 0.5 + 0.5*(score/abs(min_score))
            else:
                return 0.5 - 0.5*(score/min_score)   
    
    # CASE 2: ONLY positive scores occur -----------------------------       
    elif max_score>0 and min_score>=0: 
        if max_score == min_score:
            return 1.0
        else:
            return 0.5 + 0.5*(score/max_score)
    
    # CASE 3: ONLY negative scores occur -----------------------------
    elif max_score<=0 and min_score<0: 
        if max_score == min_score:
            return 0.0
        else:
            return 0.5 - 0.5*(score/min_score)    
  
      
def getRGB (c_tuple):
    return "#%02x%02x%02x"%(int(c_tuple[0]*255), int(c_tuple[1]*255), int(c_tuple[2]*255))

     
def span_word (word, score, colormap):
    return "<span style=\"background-color:"+getRGB(colormap(score))+"\">"+word+"</span>"


def html_heatmap (words, scores, cmap_name="bwr"):
    
    colormap  = plt.get_cmap(cmap_name)
     
    assert len(words)==len(scores)
    max_s     = max(scores)
    min_s     = min(scores)
    
    output_text = ""
    
    for idx, w in enumerate(words):
        score       = rescale_score_by_abs(scores[idx], max_s, min_s)
        output_text = output_text + span_word(w, score, colormap) + " "

    return output_text + "\n"
            
relevanceScores = [-1.00 * scr for scr in wordRelevanceList]
htmlHeatMap = html_heatmap(wordList, relevanceScores)
print ("\nLRP heatmap:")    
display(HTML(htmlHeatMap))


LRP heatmap:


In [23]:
for i, inputStr in enumerate(x_test):
    inputStr = str(getWordsFromIndexList(inputStr))
    prediction, predictionScore = getPrediction(inputStr)
    if str(getSentiment(prediction)) == sentimentDict[y_test[i]]:
        pass
    else :
        print('Prediction = ', str(getSentiment(prediction)))
        print('Actual = ',sentimentDict[y_test[i]])
        with DeepExplain(session=K.get_session()) as de:  # <-- init DeepExplain context

            '''
            Need to reconstruct the graph in DeepExplain context, using the same weights.
            1. Get the input tensor
            2. Get embedding tensor
            3. Target the output of the last dense layer (pre-softmax)
            '''

            inputTensor = model.layers[0].input
            embeddingTensor = model.layers[0].output
            preSoftmax = model.layers[-2].output

            # Sample Data for attribution
            wordIndexArray = getIndexArray(inputStr)
            sampleX = pad_sequences(wordIndexArray, maxlen = maxWords, value = 0.)

            # Perform Embedding Lookup
            getEmbeddingOutput = K.function([inputTensor],[embeddingTensor])
            embeddingOutput = getEmbeddingOutput([sampleX])[0]

            # Get Prediction for attribution
            prediction, predictionScore = getPrediction(inputStr)
            ys = np_utils.to_categorical(prediction, num_classes = nb_classes)

            #relevances = de.explain('grad*input', targetTensor * ys, inputTensor, sampleX)
            #relevances = de.explain('saliency', targetTensor * ys, inputTensor, sampleX)
            #relevances = de.explain('intgrad', targetTensor * ys, inputTensor, sampleX)
            #relevances = de.explain('deeplift', targetTensor * ys, inputTensor, sampleX)
            relevances = de.explain('elrp', preSoftmax * ys, embeddingTensor, embeddingOutput)
            #relevances = de.explain('occlusion', targetTensor * ys, inputTensor, sampleX)
    #         print ("Feature Relevances Generated...")

            wordList, wordRelevanceList, normalizedRelevanceList = getWordsAndRelevances(sampleX, relevances, prediction[0])
        #     showWordRelevances(wordList, wordRelevanceList, normalizedRelevanceList)

            relevanceScores = [-1.00 * scr for scr in wordRelevanceList]
            htmlHeatMap = html_heatmap(wordList, relevanceScores)
    #         print ("\nLRP heatmap:")    
            display(HTML(htmlHeatMap))

Prediction =  changeseatassignment
Actual =  getseatinfo


Prediction =  changeseatassignment
Actual =  getseatinfo


Prediction =  outofdomain
Actual =  rejection
sorry -0.5
sorry 1.4671486616134644


Prediction =  contentonly
Actual =  confirmation
ok 0.5
ok -1.7738949060440063


Prediction =  changeseatassignment
Actual =  outofdomain


...

Prediction =  confirmation
Actual =  contentonly
ok 0.5
ok -0.5


Prediction =  contentonly
Actual =  changeseatassignment


Prediction =  rejection
Actual =  closinggreeting


Prediction =  rejection
Actual =  closinggreeting


Prediction =  confirmation
Actual =  contentonly
ok 0.5
ok -0.5
